# LP2 CUSTOMER CHURN PROJECT

<B>PROJECT DESCRIPTION<b>
#### Customer Churn is the percentage of customers that stopped using a companys' product or service during a certain time frame.In business, understanding the primary cause of a customer churn can assist businesses to create a retention strategy to reduce customer churn and as such boost revenue.This project seeks to build a powerful machine learning pipeline that will estimate or predict the likelihood of a customer leaving vodafone or not.
In this Project, I will: 

1. classify models and help the client, a telecommunication company, to understand their data.
2. Find the lifetime value of each customer and know what factors affect the rate at which customers stop using their network.
3. Predict if a customer will churn or not.

<b> project Workflow<b>

a. hypothesis

b. research questions

c. import 

d. create variable environment

d. load data

# 1. HYPOTHESIS 

<B>NULL HYPOTHESIS:<B>

the number of months the customer has stayed in the company does not affect their churn rate

<B>ALTERNATIVE HYPOTHESIS:<B>

the number of months the customer has stayed in the company  affect their churn rate

# importation 

In [33]:
#import 
import numpy as np
import pandas as pd
import pydoc
import pyodbc 
import sqlalchemy 
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as stat
from scipy import stats
from dotenv import dotenv_values
import warnings
import os

warnings.filterwarnings('ignore')





<b>load data

In [37]:
# Load environment variables from .env file into a dictionary
environment_variable=dotenv_values('.env')

# Get the values for the credentials you set in the '.env' file
server = environment_variable.get('SERVER')
database= environment_variable.get('DATABASE')
username = environment_variable.get('USERNAME')
password = environment_variable.get('PASSWORD')

# a connection for server, database, username and password from the env 
connection_string = f"DRIVER= {{SQL Server}}; SERVER ={server}; DATABASE={database}, UID={username}, PWD ={password}  MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"



In [36]:
# connection to the server
connection=pyodbc.connect(connection_string)

OperationalError: ('08001', '[08001] [Microsoft][ODBC SQL Server Driver]Neither DSN nor SERVER keyword supplied (0) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver]Invalid connection string attribute (0)')